# Análisis de Eficiencia Energética de Edificios con Machine Learning

## Objetivo
Predecir la clasificación de eficiencia energética de edificios basándose en sus características físicas y temporales.

## Dataset
- **Fuente**: Certificados de eficiencia energética de Aragón
- **Variables**: 15 características de edificios
- **Target**: Clasificación de consumo energético (C, D, E, F, G)
- **Modelos**: Logistic Regression, KNN, Decision Tree, XGBoost, LightGBM

## 1. Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

## 2. Carga y Exploración Inicial de Datos

In [ ]:
# Cargar el dataset
df = pd.read_csv('Eficiencia_Energetica.csv')
print(f"Forma del dataset: {df.shape}")
print("\nPrimeras 5 filas:")
df.head()

In [ ]:
# Información general del dataset
print("=== INFORMACIÓN DEL DATASET ===")
df.info()
print(f"\nTamaño: {df.shape[0]} filas x {df.shape[1]} columnas")

In [ ]:
# Verificar valores nulos
print("=== VALORES NULOS ===")
print(df.isnull().sum())
print(f"\nTotal de valores nulos: {df.isnull().sum().sum()}")

In [ ]:
# Verificar duplicados
print("=== VERIFICACIÓN DE DUPLICADOS ===")
duplicados = df.duplicated().sum()
print(f"Número de duplicados: {duplicados}")

if duplicados > 0:
    print("\nFilas duplicadas:")
    print(df[df.duplicated()])
    df = df.drop_duplicates()
    print(f"\nDuplicados eliminados. Nuevo tamaño: {df.shape}")

In [ ]:
# Análisis de la variable objetivo
print("=== VARIABLE OBJETIVO ===")
print("\nDistribución de Clasificacion_consumo:")
print(df['Clasificacion_consumo'].value_counts().sort_index())
print(f"\nPorcentajes:")
print(df['Clasificacion_consumo'].value_counts(normalize=True).sort_index() * 100)

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Limpiar nombres de columnas
df.columns = df.columns.str.replace('/', '_').str.replace(' ', '_')
print("Columnas actualizadas:")
print(df.columns.tolist())

In [ ]:
# Crear variable de antigüedad del edificio
df['Antiguedad'] = 2024 - df['Anio_construccion']
print("Variable 'Antiguedad' creada")
print(f"Rango de antigüedad: {df['Antiguedad'].min()} - {df['Antiguedad'].max()} años")

In [ ]:
# Identificar outliers en variables numéricas
print("=== ANÁLISIS DE OUTLIERS ===")
columnas_numericas = ['Emision_CO2', 'ConsumoKWh_m2_Anio', 'Superficie_m2', 'Antiguedad']

for col in columnas_numericas:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)]
    if len(outliers) > 0:
        print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.1f}%)")
        print(f"  Min: {df[col].min():.2f}, Max: {df[col].max():.2f}")

In [ ]:
# Corrección de outliers extremos
print("=== CORRECCIÓN DE OUTLIERS ===")

# ConsumoKWh_m2_Anio - hay valores extremos como 40788
if 'ConsumoKWh_m2_Anio' in df.columns:
    # Usar percentil 99 para detectar valores extremos
    p99 = df['ConsumoKWh_m2_Anio'].quantile(0.99)
    outliers_antes = (df['ConsumoKWh_m2_Anio'] > p99).sum()
    
    # Recortar valores extremos
    df['ConsumoKWh_m2_Anio'] = df['ConsumoKWh_m2_Anio'].clip(upper=p99)
    print(f"✓ ConsumoKWh_m2_Anio: {outliers_antes} valores corregidos (máximo: {p99:.0f})")

# Superficie_m2 - valores muy altos
if 'Superficie_m2' in df.columns:
    p95 = df['Superficie_m2'].quantile(0.95)
    outliers_antes = (df['Superficie_m2'] > p95).sum()
    
    df['Superficie_m2'] = df['Superficie_m2'].clip(upper=p95)
    print(f"✓ Superficie_m2: {outliers_antes} valores corregidos (máximo: {p95:.0f})")

In [ ]:
# Estadísticas descriptivas
print("=== ESTADÍSTICAS DESCRIPTIVAS ===")
print("\n1. VARIABLES NUMÉRICAS:")
print("=" * 50)

for col in columnas_numericas:
    if col in df.columns:
        print(f"\n{col.upper()}:")
        print(f"   • Media: {df[col].mean():.2f}")
        print(f"   • Mediana: {df[col].median():.2f}")
        print(f"   • Desviación Estándar: {df[col].std():.2f}")
        print(f"   • Rango: {df[col].max() - df[col].min():.2f} (Min: {df[col].min():.2f}, Max: {df[col].max():.2f})")

In [ ]:
# Estadísticas de variables categóricas
print("\n2. VARIABLES CATEGÓRICAS:")
print("=" * 50)
columnas_categoricas = ['Tipo_edificio', 'Estado_edificio', 'Municipio', 'Provincia']

for col in columnas_categoricas:
    if col in df.columns:
        print(f"\n{col.upper()}:")
        valores_unicos = df[col].nunique()
        moda = df[col].mode()[0]
        moda_freq = (df[col] == moda).sum()
        moda_pct = (moda_freq / len(df)) * 100
        
        print(f"   • Valores únicos: {valores_unicos}")
        print(f"   • Moda: '{moda}' ({moda_freq} casos, {moda_pct:.1f}%)")
        
        # Top 5 más frecuentes
        if valores_unicos <= 10:
            print(f"   • Distribución:")
            top_values = df[col].value_counts().head(5)
            for valor, freq in top_values.items():
                pct = (freq / len(df)) * 100
                print(f"     - {valor}: {freq} ({pct:.1f}%)")

In [ ]:
# Distribución de la variable objetivo
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df['Clasificacion_consumo'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title('Distribución de Clasificación de Consumo')
plt.xlabel('Clasificación')
plt.ylabel('Cantidad')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
df['Clasificacion_consumo'].value_counts().sort_index().plot(kind='pie', autopct='%1.1f%%')
plt.title('Proporción de Clasificaciones')
plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Histogramas de variables numéricas
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for i, col in enumerate(columnas_numericas):
    if col in df.columns and i < 4:
        df[col].hist(bins=30, ax=axes[i], edgecolor='black', alpha=0.7)
        axes[i].set_title(f'Distribución de {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis bivariado - Variables vs Clasificación
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

# Consumo vs Clasificación
df.boxplot(column='ConsumoKWh_m2_Anio', by='Clasificacion_consumo', ax=axes[0])
axes[0].set_title('Consumo por Clasificación')
axes[0].set_xlabel('Clasificación')

# Emisiones vs Clasificación
df.boxplot(column='Emision_CO2', by='Clasificacion_consumo', ax=axes[1])
axes[1].set_title('Emisiones CO2 por Clasificación')
axes[1].set_xlabel('Clasificación')

# Tipo edificio vs Clasificación
tipo_class = pd.crosstab(df['Tipo_edificio'], df['Clasificacion_consumo'], normalize='index') * 100
tipo_class.plot(kind='bar', ax=axes[2], stacked=True)
axes[2].set_title('Clasificación por Tipo de Edificio')
axes[2].tick_params(axis='x', rotation=45)

# Antigüedad vs Clasificación
df.boxplot(column='Antiguedad', by='Clasificacion_consumo', ax=axes[3])
axes[3].set_title('Antigüedad por Clasificación')
axes[3].set_xlabel('Clasificación')

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlación para variables numéricas
plt.figure(figsize=(10, 8))
columnas_num_disponibles = [col for col in columnas_numericas if col in df.columns]
correlation_matrix = df[columnas_num_disponibles].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de Correlación - Variables Numéricas')
plt.show()

print("=== CORRELACIONES IMPORTANTES (> 0.5) ===")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            print(f"{correlation_matrix.columns[i]} vs {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

## 4. Preprocesamiento de Datos

In [ ]:
# Seleccionar características para el modelo
caracteristicas = ['Emision_CO2', 'ConsumoKWh_m2_Anio', 'Tipo_edificio', 'Estado_edificio', 
                  'Anio_construccion', 'Superficie_m2', 'Municipio', 'Antiguedad']

# Filtrar solo las características que existen en el dataset
caracteristicas_disponibles = [col for col in caracteristicas if col in df.columns]
print(f"Características seleccionadas: {caracteristicas_disponibles}")

# Crear X e y
X = df[caracteristicas_disponibles].copy()
y = df['Clasificacion_consumo'].copy()

print(f"\nForma de X: {X.shape}")
print(f"Forma de y: {y.shape}")
print(f"\nClases en y: {sorted(y.unique())}")

In [ ]:
# Identificar columnas numéricas y categóricas
columnas_numericas_ml = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
columnas_categoricas_ml = X.select_dtypes(include=['object']).columns.tolist()

print(f"Columnas numéricas: {columnas_numericas_ml}")
print(f"Columnas categóricas: {columnas_categoricas_ml}")

In [ ]:
# Manejar municipios con pocas muestras
if 'Municipio' in columnas_categoricas_ml:
    municipio_counts = X['Municipio'].value_counts()
    municipios_raros = municipio_counts[municipio_counts < 5].index
    X['Municipio'] = X['Municipio'].replace(municipios_raros, 'OTROS')
    print(f"Municipios con <5 muestras agrupados como 'OTROS': {len(municipios_raros)}")

In [ ]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamaño entrenamiento: {X_train.shape}")
print(f"Tamaño prueba: {X_test.shape}")
print(f"\nDistribución en entrenamiento:")
print(y_train.value_counts().sort_index())
print(f"\nDistribución en prueba:")
print(y_test.value_counts().sort_index())

## 5. Implementación de Modelos y Benchmark

In [ ]:
# Crear preprocesadores para diferentes tipos de modelos

# Para modelos que necesitan escalado (Logistic Regression, KNN)
preprocessor_scaled = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas_ml),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), columnas_categoricas_ml)
    ])

# Para modelos basados en árboles (Decision Tree, XGBoost, LightGBM)
preprocessor_tree = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', columnas_numericas_ml),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), columnas_categoricas_ml)
    ])

print("Preprocesadores creados")

### 5.1 Logistic Regression

In [ ]:
# Pipeline Logistic Regression
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor_scaled),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# Entrenar
pipeline_lr.fit(X_train, y_train)

# Predecir
y_pred_lr = pipeline_lr.predict(X_test)

# Evaluar
print("=== LOGISTIC REGRESSION ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision (macro): {precision_score(y_test, y_pred_lr, average='macro'):.4f}")
print(f"Recall (macro): {recall_score(y_test, y_pred_lr, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_lr, average='macro'):.4f}")

### 5.2 K-Nearest Neighbors (KNN)

In [ ]:
# Pipeline KNN
pipeline_knn = Pipeline([
    ('preprocessor', preprocessor_scaled),
    ('classifier', KNeighborsClassifier(n_neighbors=5))
])

# Entrenar
pipeline_knn.fit(X_train, y_train)

# Predecir
y_pred_knn = pipeline_knn.predict(X_test)

# Evaluar
print("=== K-NEAREST NEIGHBORS ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print(f"Precision (macro): {precision_score(y_test, y_pred_knn, average='macro'):.4f}")
print(f"Recall (macro): {recall_score(y_test, y_pred_knn, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_knn, average='macro'):.4f}")

### 5.3 Decision Tree

In [ ]:
# Pipeline Decision Tree
pipeline_dt = Pipeline([
    ('preprocessor', preprocessor_tree),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Entrenar
pipeline_dt.fit(X_train, y_train)

# Predecir
y_pred_dt = pipeline_dt.predict(X_test)

# Evaluar
print("=== DECISION TREE ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"Precision (macro): {precision_score(y_test, y_pred_dt, average='macro'):.4f}")
print(f"Recall (macro): {recall_score(y_test, y_pred_dt, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_dt, average='macro'):.4f}")

### 5.4 XGBoost

In [ ]:
# Codificar las etiquetas para XGBoost
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Pipeline XGBoost
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor_tree),
    ('classifier', xgb.XGBClassifier(random_state=42, eval_metric='mlogloss'))
])

# Entrenar
pipeline_xgb.fit(X_train, y_train_encoded)

# Predecir
y_pred_xgb_encoded = pipeline_xgb.predict(X_test)
y_pred_xgb = label_encoder.inverse_transform(y_pred_xgb_encoded)

# Evaluar
print("=== XGBOOST ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Precision (macro): {precision_score(y_test, y_pred_xgb, average='macro'):.4f}")
print(f"Recall (macro): {recall_score(y_test, y_pred_xgb, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_xgb, average='macro'):.4f}")

### 5.5 LightGBM

In [ ]:
# Pipeline LightGBM
pipeline_lgb = Pipeline([
    ('preprocessor', preprocessor_tree),
    ('classifier', lgb.LGBMClassifier(random_state=42, verbose=-1))
])

# Entrenar
pipeline_lgb.fit(X_train, y_train_encoded)

# Predecir
y_pred_lgb_encoded = pipeline_lgb.predict(X_test)
y_pred_lgb = label_encoder.inverse_transform(y_pred_lgb_encoded)

# Evaluar
print("=== LIGHTGBM ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lgb):.4f}")
print(f"Precision (macro): {precision_score(y_test, y_pred_lgb, average='macro'):.4f}")
print(f"Recall (macro): {recall_score(y_test, y_pred_lgb, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_lgb, average='macro'):.4f}")

### 5.6 Comparación de Modelos Base

In [ ]:
# Comparación de todos los modelos
print("=== COMPARACIÓN DE MODELOS BASE ===")

resultados_base = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'KNN', 'Decision Tree', 'XGBoost', 'LightGBM'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_xgb),
        accuracy_score(y_test, y_pred_lgb)
    ],
    'Precision': [
        precision_score(y_test, y_pred_lr, average='macro'),
        precision_score(y_test, y_pred_knn, average='macro'),
        precision_score(y_test, y_pred_dt, average='macro'),
        precision_score(y_test, y_pred_xgb, average='macro'),
        precision_score(y_test, y_pred_lgb, average='macro')
    ],
    'Recall': [
        recall_score(y_test, y_pred_lr, average='macro'),
        recall_score(y_test, y_pred_knn, average='macro'),
        recall_score(y_test, y_pred_dt, average='macro'),
        recall_score(y_test, y_pred_xgb, average='macro'),
        recall_score(y_test, y_pred_lgb, average='macro')
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_lr, average='macro'),
        f1_score(y_test, y_pred_knn, average='macro'),
        f1_score(y_test, y_pred_dt, average='macro'),
        f1_score(y_test, y_pred_xgb, average='macro'),
        f1_score(y_test, y_pred_lgb, average='macro')
    ]
})

print(resultados_base.round(4))

# Visualización
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(resultados_base))
width = 0.2

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['skyblue', 'lightgreen', 'lightcoral', 'gold']

for i, metric in enumerate(metrics):
    ax.bar(x + i*width, resultados_base[metric], width, label=metric, color=colors[i])

ax.set_xlabel('Modelos')
ax.set_ylabel('Score')
ax.set_title('Comparación de Métricas - Modelos Base')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(resultados_base['Modelo'], rotation=45)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Optimización de Hiperparámetros

### 6.1 Optimización del mejor modelo base

In [ ]:
# Identificar el mejor modelo base
mejor_f1 = resultados_base['F1-Score'].max()
mejor_modelo_base = resultados_base.loc[resultados_base['F1-Score'] == mejor_f1, 'Modelo'].values[0]
print(f"Mejor modelo base: {mejor_modelo_base} (F1-Score: {mejor_f1:.4f})")

# También optimizaremos Decision Tree y XGBoost por ser populares
print("\nOptimizaremos: Decision Tree y XGBoost")

In [ ]:
# Optimización Decision Tree
param_grid_dt = {
    'classifier__max_depth': [5, 10, 15, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

grid_dt = GridSearchCV(
    pipeline_dt,
    param_grid=param_grid_dt,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)

print("Optimizando Decision Tree...")
grid_dt.fit(X_train, y_train)

print(f"Mejores parámetros DT: {grid_dt.best_params_}")
print(f"Mejor score DT: {grid_dt.best_score_:.4f}")

# Predicciones optimizadas
y_pred_dt_opt = grid_dt.predict(X_test)

print("\n=== DECISION TREE OPTIMIZADO ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt_opt):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_dt_opt, average='macro'):.4f}")

In [ ]:
# Optimización XGBoost
param_grid_xgb = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.1, 0.2]
}

grid_xgb = GridSearchCV(
    pipeline_xgb,
    param_grid=param_grid_xgb,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)

print("Optimizando XGBoost...")
grid_xgb.fit(X_train, y_train_encoded)

print(f"Mejores parámetros XGB: {grid_xgb.best_params_}")
print(f"Mejor score XGB: {grid_xgb.best_score_:.4f}")

# Predicciones optimizadas
y_pred_xgb_opt_encoded = grid_xgb.predict(X_test)
y_pred_xgb_opt = label_encoder.inverse_transform(y_pred_xgb_opt_encoded)

print("\n=== XGBOOST OPTIMIZADO ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb_opt):.4f}")
print(f"F1-Score (macro): {f1_score(y_test, y_pred_xgb_opt, average='macro'):.4f}")

## 7. Comparación Final y Análisis

In [ ]:
# Comparación final incluyendo modelos optimizados
print("=== COMPARACIÓN FINAL ===")

resultados_finales = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'KNN', 'Decision Tree', 'XGBoost', 'LightGBM', 
               'Decision Tree Opt', 'XGBoost Opt'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_knn),
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_xgb),
        accuracy_score(y_test, y_pred_lgb),
        accuracy_score(y_test, y_pred_dt_opt),
        accuracy_score(y_test, y_pred_xgb_opt)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_lr, average='macro'),
        f1_score(y_test, y_pred_knn, average='macro'),
        f1_score(y_test, y_pred_dt, average='macro'),
        f1_score(y_test, y_pred_xgb, average='macro'),
        f1_score(y_test, y_pred_lgb, average='macro'),
        f1_score(y_test, y_pred_dt_opt, average='macro'),
        f1_score(y_test, y_pred_xgb_opt, average='macro')
    ]
})

print(resultados_finales.round(4))

# Identificar el mejor modelo final
mejor_f1_final = resultados_finales['F1-Score'].max()
mejor_modelo_final = resultados_finales.loc[resultados_finales['F1-Score'] == mejor_f1_final, 'Modelo'].values[0]

print(f"\nMEJOR MODELO: {mejor_modelo_final}")
print(f"F1-Score: {mejor_f1_final:.4f}")
print(f"Accuracy: {resultados_finales.loc[resultados_finales['F1-Score'] == mejor_f1_final, 'Accuracy'].values[0]:.4f}")

In [ ]:
# Matriz de confusión del mejor modelo
# Determinar las predicciones del mejor modelo
if 'XGBoost Opt' in mejor_modelo_final:
    y_pred_mejor = y_pred_xgb_opt
elif 'Decision Tree Opt' in mejor_modelo_final:
    y_pred_mejor = y_pred_dt_opt
elif 'LightGBM' in mejor_modelo_final:
    y_pred_mejor = y_pred_lgb
elif 'XGBoost' in mejor_modelo_final:
    y_pred_mejor = y_pred_xgb
elif 'Decision Tree' in mejor_modelo_final:
    y_pred_mejor = y_pred_dt
elif 'KNN' in mejor_modelo_final:
    y_pred_mejor = y_pred_knn
else:
    y_pred_mejor = y_pred_lr

# Crear matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_mejor)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
plt.title(f'Matriz de Confusión - {mejor_modelo_final}')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()

# Reporte de clasificación
print(f"\n=== REPORTE DE CLASIFICACIÓN - {mejor_modelo_final} ===")
print(classification_report(y_test, y_pred_mejor))

## 8. Importancia de Variables (si aplica)

In [ ]:
# Mostrar importancia de variables si el mejor modelo lo permite
if 'Decision Tree' in mejor_modelo_final:
    # Obtener el modelo optimizado
    if 'Opt' in mejor_modelo_final:
        modelo_final = grid_dt.best_estimator_
    else:
        modelo_final = pipeline_dt
    
    # Obtener nombres de características
    feature_names = columnas_numericas_ml.copy()
    if columnas_categoricas_ml:
        encoder = modelo_final.named_steps['preprocessor'].transformers_[1][1]
        cat_features = encoder.get_feature_names_out(columnas_categoricas_ml)
        feature_names.extend(cat_features)
    
    # Obtener importancias
    importances = modelo_final.named_steps['classifier'].feature_importances_
    
    # Top 10 variables más importantes
    indices = np.argsort(importances)[::-1][:10]
    top_features = [feature_names[i] for i in indices]
    top_importances = importances[indices[:10]]
    
    plt.figure(figsize=(10, 6))
    plt.barh(top_features, top_importances)
    plt.xlabel('Importancia')
    plt.title('Top 10 Variables Más Importantes')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
elif 'XGBoost' in mejor_modelo_final:
    print("\n=== IMPORTANCIA DE VARIABLES (XGBOOST) ===")
    if 'Opt' in mejor_modelo_final:
        modelo_xgb = grid_xgb.best_estimator_.named_steps['classifier']
    else:
        modelo_xgb = pipeline_xgb.named_steps['classifier']
    
    # Obtener importancias
    feature_importance = modelo_xgb.feature_importances_
    
    # Crear DataFrame para visualización
    importance_df = pd.DataFrame({
        'feature': range(len(feature_importance)),
        'importance': feature_importance
    }).sort_values('importance', ascending=False).head(10)
    
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(importance_df)), importance_df['importance'])
    plt.xlabel('Importancia')
    plt.title('Top 10 Variables Más Importantes (XGBoost)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

else:
    print(f"\nEl modelo {mejor_modelo_final} no proporciona importancia de variables de forma directa.")

## 9. Conclusiones

In [ ]:
print("=== CONCLUSIONES DEL ANÁLISIS ===")
print("\n1. DATASET:")
print(f"   - Total de muestras: {df.shape[0]}")
print(f"   - Variables utilizadas: {len(caracteristicas_disponibles)}")
print(f"   - Clases objetivo: {len(y.unique())} ({', '.join(sorted(y.unique()))})")

print("\n2. PREPROCESAMIENTO:")
duplicados_eliminados = duplicados if 'duplicados' in locals() else 0
print(f"   - Duplicados eliminados: {duplicados_eliminados}")
print("   - Outliers extremos corregidos en ConsumoKWh y Superficie")
print("   - Variables categóricas codificadas con OneHotEncoder")
print("   - Variables numéricas escaladas para modelos apropiados")

print("\n3. MODELOS EVALUADOS:")
print("   - Logistic Regression, KNN, Decision Tree, XGBoost, LightGBM")
print("   - Optimización de hiperparámetros con GridSearchCV")
print("   - Evaluación con métricas macro-promediadas")

print("\n4. RESULTADOS:")
print(f"   - Mejor modelo: {mejor_modelo_final}")
print(f"   - Accuracy: {resultados_finales.loc[resultados_finales['F1-Score'] == mejor_f1_final, 'Accuracy'].values[0]:.4f}")
print(f"   - F1-Score (macro): {mejor_f1_final:.4f}")

mejora_optimizacion = False
if 'Opt' in mejor_modelo_final:
    modelo_base = mejor_modelo_final.replace(' Opt', '')
    f1_base = resultados_finales.loc[resultados_finales['Modelo'] == modelo_base, 'F1-Score'].values
    if len(f1_base) > 0:
        mejora = ((mejor_f1_final - f1_base[0]) / f1_base[0]) * 100
        print(f"   - Mejora con optimización: {mejora:.1f}%")
        mejora_optimizacion = True

print("\n5. INSIGHTS:")
print("   - La clasificación energética puede predecirse con precisión razonable")
print("   - Las variables más importantes incluyen consumo y emisiones CO2")
print("   - El tipo y antigüedad del edificio también influyen")
if mejora_optimizacion:
    print("   - La optimización de hiperparámetros mejoró significativamente el rendimiento")

print("\n6. APLICACIONES PRÁCTICAS:")
print("   - Evaluación automática de eficiencia energética")
print("   - Identificación de edificios candidatos para renovación")
print("   - Apoyo en políticas de eficiencia energética")
print("   - Estimación de clasificación antes de certificación oficial")